In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

# Open the data file or files

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

##josie laptop 
#topdir = "/Users/josieswann/Desktop/important documents"

# On Bellis' laptop
#topdir = '/home/bellis/babar_data/bnv_plambda/'


# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")
IS_MC=True


#'''
# Collision data
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'

start = time.time()

data_collision = ak.from_parquet(filename)
#data_collision = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data_collision))
#'''


In [ ]:
# Run over collision data
#data = data_collision
#IS_MC=False

## Add an entry with the tag-side B variable(s)

In [ ]:
bat.fill_new_entry_with_tag_side_B(data)
data['BtagSideMes']

In [ ]:
bat.fill_new_entry_with_tag_side_B(data_collision)
data_collision['BtagSideMes']

# Get information about cross sections and numbers of events

In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

# Make a few test histograms 

In [ ]:
sp = data['spmode']

np.unique(sp.to_list())

In [ ]:
region_definitions

In [ ]:
hist_defs

# Create standard empty histograms

In [ ]:
# Make our histograms
all_hists = bat.create_empty_histograms(hist_defs)

bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)
print()
print(spmodes)

In [ ]:
# Pull out some SP mode
#spmode = '1005'
spmode='998'
#spmode='991'
#spmode='-999'
#spmode='0'
mask_sp = data['spmode'] == spmode

# Collision data
#mask_sp = data['spmode']=='0'

# Make a subset
data_sp = data[mask_sp]

# For data we need to account for blinding
if IS_MC is False:
    nB = data_sp['nB']
    Bp3 = data_sp['Bp3']
    nBp3 = ak.num(Bp3)
    
    mask_to_account_for_blinding = nB == nBp3
    print(len(nB[mask_to_account_for_blinding]), len(nB[~mask_to_account_for_blinding]))
    
    data_sp = data_sp[mask_to_account_for_blinding]

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

# Make the plot
mes =    ak.flatten(data_sp['BpostFitMes'][fit_mask])
DeltaE = ak.flatten(data_sp['BpostFitDeltaE'][fit_mask])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp['Lambda0_unc_Mass'][fit_mask])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

In [ ]:
#mes

# Get the event-based cuts



In [ ]:
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

In [ ]:
################################################################################
# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

#spmode = '998'
#spmode = '1005'
spmode = '-999'
#spmode = '1235'

mask_sp = data['spmode']==spmode

mask = mask_sp & mask_event
################################################################################

# Make the plot
mes =    ak.flatten(data[mask]['BpostFitMes'])
DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True, bins=25)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data[mask]['Lambda0_unc_Mass'])
weight = 1.0#weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

# Write out the values

## parquet

In [ ]:
################################################################################
# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

#spmode = '998'
#spmode = '1005'
spmode = '-999'
#spmode = '1235'

mask_sp = data['spmode']==spmode

mask = mask_sp & mask_event
################################################################################

# Make the plot
mes =    ak.flatten(data[mask]['BpostFitMes'])
DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])
out_array = ak.Array({'mes': mes, 'DeltaE': DeltaE})

output_filename = f'mes_deltae_{spmode}_AFTER_FINAL_CUTS.parquet'
ak.to_parquet(out_array, f'{output_filename}.parquet')


In [ ]:
print(f"Opening {output_filename}.parquet")

filename = f'{output_filename}.parquet'
#filename = f'bnv_plambda/Background_SP_modes_Only_Run_1.parquet'

start = time.time()

data_test = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

In [ ]:
data_test

In [ ]:
data_test['mes']

## dataframe

In [ ]:
################################################################################
# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

tag = "FINAL_CUTS"
tag = "EARLY_CUT"

mask = mask_event
################################################################################
subset = ['spmode', 'BpostFitMes', 'BpostFitDeltaE', 'Lambda0_unc_Mass', \
          'BtagSideMes', 'BSphr', 'BThrust', 'BCosThetaS', \
          'R2', 'R2All', \
          'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
          'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
          'BThrustROE']

ak_array_type = type(data['spmode'])

df_dict = {}
for var in subset:
    x = data[mask][var]

    # If this is nested, then flatten it
    if type(x[0]) == ak_array_type:
        x = ak.flatten(data[mask][var])
        
    df_dict[var] = x
# Make the plot
df_out = pd.DataFrame.from_dict(df_dict)

# Write it
outfilename = "output_variables_{tag}.parquet"
df_out.to_parquet(outfilename)


df_out

In [ ]:
df_in = pd.read_parquet(outfilename)

df_in

In [ ]:
#sns.histplot(data=df_in, x="BpostFitMes", hue='spmode', bins=100, binrange=(5.2,5.3), stat='probability')

# Older backup development code

In [ ]:
#mask_sp = data['spmode']=='998'
#mask_nB, mask_duplicates= bat.get_duplicates_mask(data[mask_sp])

mask_lambda0, mask_event_nlambda0_and_nB = bat.get_lambda0_mask(data_sp, region_definitions=region_definitions, \
                                                                flightlenvar='Lambda0FlightLen')

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

######################################

'''
mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data_sp, \
              lamp_selector='SuperLooseKMProtonSelection', \
              lampi_selector='VeryTightKMPionMicroSelection', \
              Bp_selector='SuperTightKMProtonSelection', \
              verbosity=0)

print(mask_bool_proton)
print(mask_bool_pion)
print(mask_bool_protonB)
'''

mask_event = mask_event_nlambda0_and_nB

#mask_pid =      mask_bool_proton[mask_event] & \
#                mask_bool_pion[mask_event] & \
#                mask_bool_protonB[mask_event]

#mask_candidates = fit_mask[mask_event] & mask_lambda0[mask_event] & mask_pid 
mask_candidates = fit_mask[mask_event] & mask_lambda0[mask_event]# & mask_pid 

# Make the plot
mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates])
DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates])

######################################

'''
mask_candidates = fit_mask & mask_lambda0
mask_event = mask_event_nlambda0

# Make the plot
mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates[mask_event]])
DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates[mask_event]])
'''

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
#'''
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp[mask_event]['Lambda0_unc_Mass'][mask_candidates])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();
#'''

### Cut on PID for protons and pions



In [ ]:
# Cut on Lambda and number of candidates
mask_lambda0, mask_event_nlambda0_and_nB = bat.get_lambda0_mask(data_sp, region_definitions=region_definitions, \
                                                                flightlenvar='Lambda0FlightLen')

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

######################################

mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data_sp, \
              lamp_selector='SuperLooseKMProtonSelection', \
              lampi_selector='VeryTightKMPionMicroSelection', \
              Bp_selector='SuperTightKMProtonSelection', \
              verbosity=0)

#print(mask_bool_proton)
#print(mask_bool_pion)
#print(mask_bool_protonB)

mask_event = mask_event_nlambda0_and_nB

mask_pid =      mask_bool_proton[mask_event] & \
                mask_bool_pion[mask_event] & \
                mask_bool_protonB[mask_event]

mask_candidates = fit_mask[mask_event] & mask_lambda0[mask_event] & mask_pid 

# Make the plot
mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates])
DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates])

######################################

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp[mask_event]['Lambda0_unc_Mass'][mask_candidates])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();


### Anti-cut on opposite-charged proton


In [ ]:
pps = myPIDselector.PIDselector("p")

# To test
selectors_to_test = ['SuperLooseKMProtonSelection',
 'VeryLooseKMProtonSelection',
 'LooseKMProtonSelection',
 'TightKMProtaonSelection',
 'VeryTightKMProtonSelection',
 'SuperTightKMProtonSelection']

selector_to_test = "TightKMProtonSelection"

# Is MC?
mask_no_antiprotons, ct = bat.build_antiproton_antimask(data_sp, pps, selector_to_test, IS_MC=IS_MC, verbose=0)

print(mask_no_antiprotons)
print(ct)
print(len(mask_no_antiprotons))
print(len(ct))


In [ ]:
# Use it
# Cut on Lambda and number of candidates
mask_lambda0, mask_event_nlambda0_and_nB = bat.get_lambda0_mask(data_sp, region_definitions=region_definitions, \
                                                                flightlenvar='Lambda0FlightLen')

# Use only the data in the fitting region
fit_mask = bat.get_fit_mask(data_sp, region_definitions)

######################################

mask_bool_proton, mask_bool_pion, mask_bool_protonB = bat.PID_masks(data_sp, \
              lamp_selector='SuperLooseKMProtonSelection', \
              lampi_selector='VeryTightKMPionMicroSelection', \
              Bp_selector='SuperTightKMProtonSelection', \
              verbosity=0)


pps = myPIDselector.PIDselector("p")
selector_to_test = "TightKMProtonSelection"
# Is MC?
mask_no_antiprotons, ct = bat.build_antiproton_antimask(data_sp, pps, selector_to_test, IS_MC=IS_MC, verbose=0)
print(f"mask no antip: {len(mask_no_antiprotons)}    mask lambda0/nB: {len(mask_event_nlambda0_and_nB)}")

mask_event = mask_event_nlambda0_and_nB & ~mask_no_antiprotons


mask_pid =      mask_bool_proton[mask_event] & \
                mask_bool_pion[mask_event] & \
                mask_bool_protonB[mask_event]

mask_candidates = fit_mask[mask_event] & mask_lambda0[mask_event] & mask_pid 

#mask_candidates = fit_mask & mask_lambda0 & mask_pid 

# Make the plot
mes =    ak.flatten(data_sp[mask_event]['BpostFitMes'][mask_candidates])
DeltaE = ak.flatten(data_sp[mask_event]['BpostFitDeltaE'][mask_candidates])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True, zoom=True, bins=25)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)
x = ak.flatten(data_sp[mask_event]['Lambda0_unc_Mass'][mask_candidates])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

In [ ]:
print(len(mes),len(DeltaE))